In [187]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
import copy

In [43]:
tf.config.experimental_run_functions_eagerly(True)

In [2]:
df = pd.read_csv("sudoku.csv")

In [31]:
df.head()

,quizzes,solutions
0,0043002090050090010700600430060020871900074000...,8643712593258497619712658434361925871986574322...
1,0401000501070039605200080000000000170009068008...,3461792581875239645296483719658324174729168358...
2,6001203840084590720000060050002640300700800069...,6951273841384596727248369158512647392739815469...
3,4972000001004000050000160986203000403009000000...,4972583161864397252537164986293815473759641828...
4,0059103080094030600275001000300002010008200070...,4659123781894735623275681497386452919548216372...


In [32]:
number_of_samples = 100000
unsolved = df['quizzes'].iloc[:number_of_samples].values
solved = df['solutions'].iloc[:number_of_samples].values

In [33]:
X = []
y = []

In [34]:
for sudoku in unsolved:
    np_solved = np.array([int(x) for x in sudoku]).reshape(9,9,1)
    X.append(np_solved)
    
for sudoku in solved:
    np_unsolved = np.array([int(y) for y in sudoku]).reshape(81,1) - 1
    y.append(np_unsolved)

In [35]:
np_X = np.array(X)
np_X = np_X/9
np_X -= 0.5
np_y = np.array(y)

In [8]:
#del(X)
#del(y)
#del(solved)
#del(unsolved)

In [36]:
np_y.shape

(100000, 81, 1)

In [37]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [38]:
#default test size = 75%
X_train, X_test, y_train, y_test = train_test_split(np_X, np_y, test_size=0.2)

In [13]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Reshape, BatchNormalization, Activation, Input,Dropout
from tensorflow.keras.optimizers import Adam

In [14]:
input_shape = (9,9,1)
batch_size = 128

In [47]:
model = Sequential()

model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same', input_shape=input_shape))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=(1,1), activation='relu', padding='same'))

model.add(Flatten())
model.add(Dense(81*9))
model.add(Reshape((-1, 9)))
model.add(Activation('softmax'))

adam = Adam(lr=0.001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 9, 9, 64)          640       
_________________________________________________________________
batch_normalization (BatchNo (None, 9, 9, 64)          256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 9, 9, 64)          36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 9, 9, 64)          256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 128)         8320      
_________________________________________________________________
flatten (Flatten)            (None, 10368)             0         
_________________________________________________________________
dense (Dense)                (None, 729)               7

In [17]:
from tensorflow.keras.callbacks import EarlyStopping

In [18]:
early_stop = EarlyStopping(monitor='val_loss',patience=2, restore_best_weights=True)

In [48]:
with tf.device('/GPU:0'):
    model.fit(X_train, y_train, epochs=10, batch_size=128, validation_data = [X_test, y_test], callbacks = [early_stop])

Train on 80000 samples, validate on 20000 samples
Epoch 1/10
80000/80000 [==============================] - 21s 266us/sample - loss: 1.3991 - accuracy: 0.4694 - val_loss: 1.4773 - val_accuracy: 0.4685
Epoch 2/10
80000/80000 [==============================] - 22s 272us/sample - loss: 0.5309 - accuracy: 0.7861 - val_loss: 0.4803 - val_accuracy: 0.7967
Epoch 3/10
80000/80000 [==============================] - 22s 279us/sample - loss: 0.4419 - accuracy: 0.8100 - val_loss: 0.4468 - val_accuracy: 0.8031
Epoch 4/10
80000/80000 [==============================] - 21s 267us/sample - loss: 0.4022 - accuracy: 0.8234 - val_loss: 0.4360 - val_accuracy: 0.8046
Epoch 5/10
80000/80000 [==============================] - 22s 274us/sample - loss: 0.3652 - accuracy: 0.8398 - val_loss: 0.4420 - val_accuracy: 0.8032
Epoch 6/10
80000/80000 [==============================] - 21s 266us/sample - loss: 0.3249 - accuracy: 0.8599 - val_loss: 0.4612 - val_accuracy: 0.7996


In [49]:
model.save("conv_model1.h5")

In [93]:
def normalize(np_a):
    
    return (np_a/9)-0.5

In [121]:
def denormalize(np_a):
    
    return (np_a+0.5)*9

In [328]:
def solve_batch(np_boards, np_solved_boards, model):
    np_results = np.empty(np_boards.shape[0],dtype=np.bool_)
    for i in np.ndindex(np_boards.shape[:1]):
        solved = solve_human_approach(np_boards[i], model)
        np_results[i] = compare_sudoku(solved, (np_solved_boards[i]+1).reshape(9,9))
        print(f"Solved sudoku number: {i}")
    correct_boards = np_results.sum()
    return correct_boards

In [330]:
def solve_human_approach (np_sample_board, model): 
    
    #np_board = np.copy(np_sample_board)
    np_board = np.copy(np_sample_board)
    #get number of blank in sudoku board
    #blanks_count = 81 - np.count_nonzero(np_board)
    
    #for _ in range(blanks_count):
    while(1):
        np_board_4d = np_board.reshape((1,9,9,1))
        predictions = model.predict(np_board_4d)[0]
        predictions_int = np.argmax(predictions, axis=1).reshape((9,9))+1           #change range of digits from <0,8> to <1,9>
        propabilities = np.around(np.max(predictions, axis=1).reshape((9,9)), 2) 
                        
        #get position with the highest propability
        np_board = denormalize(np_board).reshape((9,9))
        zeros = (np_board == 0)
        if (zeros.sum() == 0):
            break
        blanks_indices = np.where(zeros)
        blanks_indices = np.array([(i,j) for i,j in zip(blanks_indices[0], blanks_indices[1])])
        position_to_fill = blanks_indices[propabilities[zeros].argmax()]
        #fill position with the highest propability
        value_to_fill = predictions_int[position_to_fill[0], position_to_fill[1]]
        
        np_board[position_to_fill[0], position_to_fill[1]] = value_to_fill
        np_board = normalize(np_board)
    return np_board

In [198]:
def compare_sudoku (np_sudoku1, np_sudoku2):
    #True/False array
    comparision = np_sudoku1 == np_sudoku2
    #print(np.count_nonzero(comparision))
    return comparision.all()

In [331]:
x = solve_batch(X_test[:500], y_test[:500], model)
x

Solved sudoku number: (0,)
Solved sudoku number: (1,)
Solved sudoku number: (2,)
Solved sudoku number: (3,)
Solved sudoku number: (4,)
Solved sudoku number: (5,)
Solved sudoku number: (6,)
Solved sudoku number: (7,)
Solved sudoku number: (8,)
Solved sudoku number: (9,)
Solved sudoku number: (10,)
Solved sudoku number: (11,)
Solved sudoku number: (12,)
Solved sudoku number: (13,)
Solved sudoku number: (14,)
Solved sudoku number: (15,)
Solved sudoku number: (16,)
Solved sudoku number: (17,)
Solved sudoku number: (18,)
Solved sudoku number: (19,)
Solved sudoku number: (20,)
Solved sudoku number: (21,)
Solved sudoku number: (22,)
Solved sudoku number: (23,)
Solved sudoku number: (24,)
Solved sudoku number: (25,)
Solved sudoku number: (26,)
Solved sudoku number: (27,)
Solved sudoku number: (28,)
Solved sudoku number: (29,)
Solved sudoku number: (30,)
Solved sudoku number: (31,)
Solved sudoku number: (32,)
Solved sudoku number: (33,)
Solved sudoku number: (34,)
Solved sudoku number: (35,)
So

Solved sudoku number: (287,)
Solved sudoku number: (288,)
Solved sudoku number: (289,)
Solved sudoku number: (290,)
Solved sudoku number: (291,)
Solved sudoku number: (292,)
Solved sudoku number: (293,)
Solved sudoku number: (294,)
Solved sudoku number: (295,)
Solved sudoku number: (296,)
Solved sudoku number: (297,)
Solved sudoku number: (298,)
Solved sudoku number: (299,)
Solved sudoku number: (300,)
Solved sudoku number: (301,)
Solved sudoku number: (302,)
Solved sudoku number: (303,)
Solved sudoku number: (304,)
Solved sudoku number: (305,)
Solved sudoku number: (306,)
Solved sudoku number: (307,)
Solved sudoku number: (308,)
Solved sudoku number: (309,)
Solved sudoku number: (310,)
Solved sudoku number: (311,)
Solved sudoku number: (312,)
Solved sudoku number: (313,)
Solved sudoku number: (314,)
Solved sudoku number: (315,)
Solved sudoku number: (316,)
Solved sudoku number: (317,)
Solved sudoku number: (318,)
Solved sudoku number: (319,)
Solved sudoku number: (320,)
Solved sudoku 

221

In [327]:
y_test[1].reshape(9,9)

array([[4, 1, 2, 8, 7, 5, 0, 6, 3],
       [8, 3, 7, 0, 6, 4, 1, 2, 5],
       [6, 5, 0, 1, 2, 3, 4, 7, 8],
       [3, 8, 1, 2, 4, 0, 7, 5, 6],
       [7, 0, 5, 6, 1, 8, 2, 3, 4],
       [2, 4, 6, 3, 5, 7, 8, 0, 1],
       [5, 2, 3, 7, 8, 1, 6, 4, 0],
       [0, 7, 8, 4, 3, 6, 5, 1, 2],
       [1, 6, 4, 5, 0, 2, 3, 8, 7]])